In [495]:
import pandas as pd
import plotnine as pn
from plotnine import aes, ggplot
import pathlib
from datetime import datetime
from prophet import Prophet

# Import Datasets
The following datasets are used:
- cpi file as supplied by zindi
- currency as supplied by zindi
- jse data as supplied by zindi
-  Monthly credit detail excel released by the reserve bank (https://www.resbank.co.za/en/home/what-we-do/statistics/releases/selected-statistics)
- fuel prices pulled from open price engin

In [496]:
# get directory path
path = str(pathlib.Path().cwd().parent.parent.resolve())

In [497]:
cpi = pd.read_csv(path + '/data/cpi.csv')
currency = pd.read_csv(path + '/data/currency_data.csv')
credit = pd.read_excel(path + '/data/credit.xlsx', header=1)
jse = pd.read_csv(path + '/data/jse_indices.csv')
fuel = pd.read_csv(path + '/data/fuel_df.csv').drop(['Unnamed: 0'], axis=1)
sabor = pd.read_csv(path + '/data/sabor.csv')

Change dates to datetime

In [498]:
# change month to datetime format
cpi['Date'] = pd.to_datetime(cpi['Month']).dt.strftime('%Y-%m')
currency['Date'] = pd.to_datetime(currency['Date']).dt.strftime('%Y-%m')
jse['Date'] = pd.to_datetime(jse['date']).dt.strftime('%Y-%m')
credit['date'] = pd.to_datetime(credit['Date'], format='%b, %Y')
credit['Date'] = pd.to_datetime(credit['date']).dt.strftime('%Y-%m')
fuel['Date'] = pd.to_datetime(fuel['date']).dt.strftime('%Y-%m')
sabor['Date'] = pd.to_datetime(sabor['date']).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_61563/602859498.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.


Only select dates from 2022

In [499]:
# change month to datetime format
cpi_new = cpi[cpi['Date'] > '2018']
currency_new = currency[currency['Date'] > '2018']
jse_new = jse[jse['Date'] > '2018']
credit_new= credit[credit['Date'] > '2018']
fuel_new= fuel[fuel['Date'] > '2018']
sabor_new= sabor[sabor['Date'] > '2018']

In [500]:
# Add June CPI
june_cpi = pd.DataFrame({'Category': ['Headline_CPI', 'Food and nonalcoholic beverages', 'Alcoholic beverages and tobacco', 
                                      'Clothing and footwear', 'Housing and utilities', 'Household contents and services',
                                      'Health', 'Transport', 'Communication', 'Recreation and culture', 'Education',
                                      'Restaurants and hotels ', 'Miscellaneous goods and services'], 
                         'Value': [109.8, 118.3, 110.9, 104.3, 105.4, 107.7, 110.8, 112.3, 99.6, 105.3, 110.4, 110.0, 109.6]})
june_cpi['Date'] = '2023-06'

In [501]:
cpi_new = pd.concat([cpi_new, june_cpi])
cpi_new['Date'] = pd.to_datetime(cpi_new['Date']).dt.strftime('%Y-%m')

In the credit and national excels the commas in values need to be stripped and values turned in to intergers

In [502]:
def remove_commas_and_convert_to_float(value):
    return float(value.replace(',', ''))

In [503]:
credit_new = credit_new.drop(['Share of corporations as a % of total credit',
       'Share of corporations as a % of total loans & advances',
       'Share of households as a % of total credit',
       'Share of households as % of total loans & advances', 'Investments'], axis=1)

In [504]:
credit_columns = ['Instalment sale credit', 'Leasing finance',
       'Mortgage advances', 'Overdrafts', 'General loans and advances',
       'Credit card advances', 'Of which: Total to households',
       'Total loans and advances : Households',
       'Claims on the domestic private sector',
       'Total loans and advances (excl. investments & bills)',
       'Bills discounted', 'Instalment sale credit.1', 'Leasing finance.1',
       'Mortgage advances.1', 'Overdrafts.1', 'General loans and advances.1',
       'Credit card advances.1', 'Of which: Total to corporations',
       'Claims on the domestic private sector.1',
       'Total loans and advances : Corporations',]

In [505]:
credit_new[credit_columns] = credit_new[credit_columns].applymap(remove_commas_and_convert_to_float)


# Create Features

In some of the excel sheets we do not have up to date. We will there use the feature from 2 months ao as the feature for now (ie march is the predictor for July)

In [506]:
print(jse_new['Date'].max())
print(credit_new['Date'].max())
print(currency_new['Date'].max())
print(cpi_new['Date'].max())
print(fuel_new['Date'].max())
print(sabor_new['Date'].max())

2023-06
2023-06
2023-06
2023-06
2023-07
2023-06


In [507]:
jse_new['newDate'] = (pd.to_datetime(jse_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
credit_new['newDate'] = (pd.to_datetime(credit_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
currency_new['newDate'] = (pd.to_datetime(currency_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
fuel_new['newDate'] = (pd.to_datetime(fuel_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
sabor_new['newDate'] = (pd.to_datetime(sabor_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_61563/1032560082.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_61563/1032560082.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [508]:
print(jse_new['newDate'].max())
print(credit_new['newDate'].max())
print(currency_new['newDate'].max())
print(fuel_new['newDate'].max())
print(sabor_new['newDate'].max())

2023-08
2023-08
2023-08
2023-09
2023-08


In [509]:
jse_new = jse_new.drop(['Date', 'date'], axis=1)
credit_new = credit_new.drop(['Date', 'date'], axis=1)
currency_new = currency_new.drop(['Date'], axis=1)
fuel_new = fuel_new.drop(['Date', 'date'], axis=1)
sabor_new = sabor_new.drop(['Date', 'date'], axis=1)

## Get mean per Month for jse and currency

In [510]:
credit_mean = credit_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
jse_mean = jse_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
currency_mean = currency_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
fuel_new = fuel_new.groupby(['newDate']).mean().reset_index()
sabor_new = sabor_new.groupby(['newDate']).mean().reset_index()

jse_std = jse_new.groupby(['newDate']).std().reset_index().add_suffix("std")
currency_std = currency_new.groupby(['newDate']).std().reset_index().add_suffix("std")

credit_min = credit_new.groupby(['newDate']).min().reset_index().add_suffix("min")
jse_min = jse_new.groupby(['newDate']).min().reset_index().add_suffix("min")
currency_min = currency_new.groupby(['newDate']).min().reset_index().add_suffix("min")

credit_max = credit_new.groupby(['newDate']).max().reset_index().add_suffix("max")
jse_max = jse_new.groupby(['newDate']).max().reset_index().add_suffix("max")
currency_max = currency_new.groupby(['newDate']).max().reset_index().add_suffix("max")


In [511]:
credit_new = credit_mean.merge(credit_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
credit_new = credit_new.merge(credit_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
credit_new = credit_new.rename(columns={"newDatemean": "newDate"})

In [512]:
jse_new = jse_mean.merge(jse_std, left_on='newDatemean', right_on='newDatestd', how='left').drop(['newDatestd'], axis=1)
jse_new = jse_new.merge(jse_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
jse_new = jse_new.merge(jse_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
jse_new = jse_new.rename(columns={"newDatemean": "newDate"})

In [513]:
currency_new = currency_mean.merge(currency_std, left_on='newDatemean', right_on='newDatestd', how='left').drop(['newDatestd'], axis=1)
currency_new = currency_new.merge(currency_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
currency_new = currency_new.merge(currency_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
currency_new = currency_new.rename(columns={"newDatemean": "newDate"})

# Merge data together for features

In [514]:
features = jse_new.merge(credit_new, right_on='newDate', left_on='newDate', how='left')
features = features.merge(currency_new, right_on='newDate', left_on='newDate', how='left')
features = features.merge(fuel_new, right_on='newDate', left_on='newDate', how='left')
features = features.merge(sabor_new, right_on='newDate', left_on='newDate', how='left')

In [515]:
# X_train = features[features['newDate'] < "2023-04"]
# X_test =  features[features['newDate'] >= "2023-04"]
X_test = features
X_train = features

X_train = X_train.merge(cpi_new[['Date', 'Category', 'Value']], left_on='newDate', right_on='Date')

# Model
We will model one index at a time to see what works the best

In [516]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [517]:
def combined_model(X_train, X_test, month):

    cpi_all = X_train[X_train['newDate'] < month]

    cpi_pred = X_test[X_test['newDate'] == month].drop(['newDate'], axis=1)

    cpi_cat = ['Headline_CPI',
                'Food and non-alcoholic beverages',
                'Alcoholic beverages and tobacco',
                'Clothing and footwear',
                'Housing and utilities',
                'Household contents and services',
                'Transport',
                'Communication',
                'Recreation and culture',
                'Restaurants and hotels ',
                'Miscellaneous goods and services', 
                'Health', 'Education']

    lr_pred = []

    # fit hw
    for cat in cpi_cat:
        df = cpi_all[cpi_all['Category'] == cat]

        lr = LinearRegression()

        lr.fit(y=df['Value'], X=df.drop(['newDate', 'Value', 'Date', "Category"], axis=1))

        lr_pred.append(round(lr.predict(cpi_pred)[0],1))

    df_lr_results = pd.DataFrame({'cat':cpi_cat, 'pred':lr_pred})

    return df_lr_results

        

## Get April predictions

In [518]:
df_results_apr = combined_model(X_train=X_train, X_test=X_test, month='2023-04')

In [519]:
cpi_apr = cpi[cpi['Date'] == '2023-04']

In [520]:
df_results_apr = df_results_apr.merge(cpi_apr[['Category', 'Value']], left_on='cat', right_on='Category')

In [521]:
df_results_apr

,cat,pred,Category,Value
0,Headline_CPI,109.1,Headline_CPI,109.4
1,Food and non-alcoholic beverages,118.7,Food and non-alcoholic beverages,117.4
2,Alcoholic beverages and tobacco,108.9,Alcoholic beverages and tobacco,110.2
3,Clothing and footwear,104.1,Clothing and footwear,103.7
4,Housing and utilities,103.8,Housing and utilities,104.6
5,Household contents and services,108.6,Household contents and services,107.7
6,Transport,111.6,Transport,113.1
7,Communication,101.0,Communication,99.8
8,Recreation and culture,103.9,Recreation and culture,104.9
9,Restaurants and hotels,108.9,Restaurants and hotels,108.6


In [522]:
rmse_apr = mean_squared_error(df_results_apr['pred'], df_results_apr['Value'], squared=False)
print('Apr RMSE: ', rmse_apr)

Apr RMSE:  0.9575610042827964


In [523]:
df_results_apr = df_results_apr[['cat', 'pred']]
df_results_apr.columns = ['ID', 'Value']

month = "April"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_apr = df_results_apr.replace(pred_map)

df_results_apr.to_csv(path + '/submissions/apr_testing_lr.csv', index=False)

## Get May predictions

In [524]:
df_results_may = combined_model(X_train=X_train, X_test=X_test, month='2023-05')

In [525]:
cpi_may = cpi[cpi['Date'] == '2023-05']

In [526]:
df_results_may = df_results_may.merge(cpi_may[['Category', 'Value']], left_on='cat', right_on='Category')

In [527]:
df_results_may

,cat,pred,Category,Value
0,Headline_CPI,110.2,Headline_CPI,109.6
1,Food and non-alcoholic beverages,119.8,Food and non-alcoholic beverages,117.7
2,Alcoholic beverages and tobacco,110.5,Alcoholic beverages and tobacco,110.6
3,Clothing and footwear,104.1,Clothing and footwear,104.1
4,Housing and utilities,105.0,Housing and utilities,104.6
5,Household contents and services,109.1,Household contents and services,107.5
6,Transport,113.9,Transport,113.3
7,Communication,100.0,Communication,99.7
8,Recreation and culture,104.6,Recreation and culture,104.8
9,Restaurants and hotels,111.7,Restaurants and hotels,110.4


In [528]:
rmse_may = mean_squared_error(df_results_may['pred'], df_results_may['Value'], squared=False)
print('May RMSE: ', rmse_may)

May RMSE:  0.9085068053595489


In [529]:
df_results_may = df_results_may[['cat', 'pred']]
df_results_may.columns = ['ID', 'Value']

month = "May"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_may = df_results_may.replace(pred_map)

df_results_may.to_csv(path + '/submissions/may_testing_lr.csv', index=False)

## Get June predictions

In [530]:
df_results_june = combined_model(X_train=X_train, X_test=X_test, month='2023-06')

In [531]:
june_values = [109.8, 110.9, 104.3, 99.6, 110.4, 118.3, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3 ]

In [532]:
cat = ['Headline_CPI',
'Alcoholic beverages and tobacco',
'Clothing and footwear',
'Communication',
'Education',
'Food and non-alcoholic beverages',
'Health',
'Household contents and services',
'Housing and utilities',
'Miscellaneous goods and services',
'Recreation and culture',
'Restaurants and hotels ',
'Transport']

In [533]:
june_cpi = pd.DataFrame({'Category':cat, 'Value':june_values})

In [534]:
df_results_june = df_results_june.merge(june_cpi[['Category', 'Value']], left_on='cat', right_on='Category')

In [535]:
df_results_june

,cat,pred,Category,Value
0,Headline_CPI,109.0,Headline_CPI,109.8
1,Food and non-alcoholic beverages,116.7,Food and non-alcoholic beverages,118.3
2,Alcoholic beverages and tobacco,112.9,Alcoholic beverages and tobacco,110.9
3,Clothing and footwear,104.0,Clothing and footwear,104.3
4,Housing and utilities,103.9,Housing and utilities,105.4
5,Household contents and services,107.6,Household contents and services,107.7
6,Transport,109.8,Transport,112.3
7,Communication,99.2,Communication,99.6
8,Recreation and culture,104.7,Recreation and culture,105.3
9,Restaurants and hotels,111.3,Restaurants and hotels,110.0


In [536]:
rmse_June = mean_squared_error(df_results_june['pred'], df_results_june['Value'], squared=False)
print('June RMSE: ', rmse_June)

June RMSE:  1.5534081637087274


In [537]:
df_results_june

,cat,pred,Category,Value
0,Headline_CPI,109.0,Headline_CPI,109.8
1,Food and non-alcoholic beverages,116.7,Food and non-alcoholic beverages,118.3
2,Alcoholic beverages and tobacco,112.9,Alcoholic beverages and tobacco,110.9
3,Clothing and footwear,104.0,Clothing and footwear,104.3
4,Housing and utilities,103.9,Housing and utilities,105.4
5,Household contents and services,107.6,Household contents and services,107.7
6,Transport,109.8,Transport,112.3
7,Communication,99.2,Communication,99.6
8,Recreation and culture,104.7,Recreation and culture,105.3
9,Restaurants and hotels,111.3,Restaurants and hotels,110.0


In [538]:
df_results_june = df_results_june[['cat', 'pred']]
df_results_june.columns = ['ID', 'Value']

month = "June"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_june = df_results_june.replace(pred_map)

df_results_june.to_csv(path + '/submissions/june_testing_lr.csv', index=False)

# Get predictions for July

In [539]:
df_results_july = combined_model(X_train=X_train, X_test=X_test, month='2023-07')

In [540]:
df_results_july.columns = ['ID', 'Value']

In [541]:
month = 'July'

In [542]:
pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

In [543]:
df_results_july = df_results_july.replace(pred_map)

In [544]:
df_results_july.to_csv(path + '/submissions/lr_testing_july.csv', index=False)

## Get predictions for August

In [545]:
df_results_august = combined_model(X_train=X_train, X_test=X_test, month='2023-08')

In [546]:
df_results_august.columns = ['ID', 'Value']

In [547]:
month = 'August'

In [548]:
pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

In [549]:
df_results_august = df_results_august.replace(pred_map)

In [550]:
df_results_august.to_csv(path + '/submissions/lr_testing_august.csv', index=False)